# Use the VLM endpoint

In [5]:
import base64
import os

from azure.identity import DefaultAzureCredential, get_bearer_token_provider
from openai import AzureOpenAI


# Function to encode an image
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")


endpoint = "https://oai-aip-cv-ont-sdc.openai.azure.com/"
model_name = "gpt-4o-mini"
deployment = "gpt-4o-mini"
token_provider = get_bearer_token_provider(DefaultAzureCredential(), "https://cognitiveservices.azure.com/.default")
api_version = "2024-12-01-preview"

client = AzureOpenAI(
    api_version=api_version,
    azure_endpoint=endpoint,
    azure_ad_token_provider=token_provider,
)

## Ask question with a photo as context

In [13]:
images_folder = "local_data/2025_Centrum/images/"  # See other notebook on how to download an image from blob store
image_name = "da812701-a1e3-4448-b5b1-0990c27441c0.jpeg"

# Getting the Base64 string
base64_image = encode_image(os.path.join(images_folder, image_name))

prompt = "You are a sidewalk inspector for the municipality. Your task is " \
"to check whether there is any damage on the sidewalk. " \
"If so, choose one or multiple categories of damage from the following list: "

list = [
  "Local subsidence",
  "Edge damage",
  "Tree root damage",
  "Transverse crack",
  "Broken elements",
  "Unfilled borehole",
  "Hole",
  "Local raise",
  "Longitudinal crack",
  "Asphalt trench elements (HOOR)",
  "Square inspection chamber connection",
  "Surface connection",
  "Fraying",
  "Edge restraint",
  "Loose elements",
  "Failed area",
  "Joint width of elements",
  "Missing elements"
]

content = f"{prompt} {list}. Please answer with only the categories in the list. If you don't find any defects, please say so. In any case, provide an explanation."

# Query endpoint
response = client.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": content,
        },
        {
            "role": "user",
            "content": [
                { "type": "text", "text": "Find if there is any damage in this image." },
                {
                    "type": "image_url",
                    "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"},
                    "detail": "low", # reduces token usage
                },
            ],
        }
    ],
    max_tokens=4096,
    temperature=1.0,
    top_p=1.0,
    model=deployment
)

print(response.choices[0].message.content)

You are a sidewalk inspector for the municipality. Your task is to check whether there is any damage on the sidewalk. If so, choose one or multiple categories of damage from the following list:  ['Local subsidence', 'Edge damage', 'Tree root damage', 'Transverse crack', 'Broken elements', 'Unfilled borehole', 'Hole', 'Local raise', 'Longitudinal crack', 'Asphalt trench elements (HOOR)', 'Square inspection chamber connection', 'Surface connection', 'Fraying', 'Edge restraint', 'Loose elements', 'Failed area', 'Joint width of elements', 'Missing elements']. Please answer with only the categories in the list. If you don't find any defects, please say so. In any case, provide an explanation.
Based on the image provided, there appears to be several types of sidewalk damage. The relevant categories of damage are:

- **Local subsidence**: The unevenness between the two surfaces suggests that part of the sidewalk has sunk in relation to the adjacent area.
- **Edge damage**: The transition betw